In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchsummary import summary
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.datasets import mnist

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [4]:
# 使用 keras 直接載入 MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
test_images = test_images.astype('float32') / 255
# test with test data
testImages = torch.from_numpy(test_images)
testLabels = torch.from_numpy(test_labels).type(torch.LongTensor)   
verify = torch.utils.data.TensorDataset(testImages, testLabels)
verify_loader = torch.utils.data.DataLoader(verify, batch_size = 100, shuffle = False)

In [5]:
def verify_model(model, input_shape, loader):
    correct_test, total_test = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            test, labels = images.view(input_shape).to(device), labels.to(device)   # 取出 testing data 以及 labels(轉 device 的型態)
            outputs = model(test)                                                   # 將測試資料輸入至模型進行測試 (Forward propagation)                                 # 計算 loss
            
            # 計算測試資料的準確度 (correct_test / total_test)
            print(torch.max(outputs.data, 1)[1])
            print(labels)
            print("--------------")
            predicted = torch.max(outputs.data, 1)[1]                               # 取出預測的 maximum
            total_test += len(labels)                                               # 全部的 label 數 (Total number of labels)
            correct_test += (predicted == labels).float().sum()   
    val_accuracy = 100 * correct_test / float(total_test)
    print("Verified Accuracy: %f (%d/%d)" % (val_accuracy, correct_test, total_test))
    return

In [10]:
# Create CNN Model
class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()
        cnn1_out_ch = 4
        cnn2_out_ch = 4
        have_linear = False
        # Convolution 1 , input_shape=(1,28,28), output_shape=(1,26,26)
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=cnn1_out_ch, kernel_size=3, stride=1, padding=0)
        # activation
        self.relu1 = nn.ReLU() 
        # Max pool 1, output_shape=(1,13,13)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2) 
        # Convolution 2, output_shape=(1,11,11)
        self.cnn2 = nn.Conv2d(in_channels=cnn1_out_ch, out_channels=cnn2_out_ch, kernel_size=3, stride=1, padding=0)
        # activation
        self.relu2 = nn.ReLU() 
        # Convolution 3, output_shape=(1,9,9)
        self.cnn3 = nn.Conv2d(in_channels=cnn2_out_ch, out_channels=10, kernel_size=3, stride=1, padding=0)
        # activation
        self.relu3 = nn.ReLU() 
        # Max pool 2, output_shape=(10,4,4)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        # Average pool, output_shape=(10,1,1)
        self.avgpool = nn.AvgPool2d(kernel_size=4)
        # Fully connected 1, input_shape=(1*5*5)
        # self.fc1 = nn.Linear(10 * 5 * 5, 10) 
        if have_linear:
            self.fc1 = nn.Linear(10 * 1 * 1, 10) 
    
    def forward(self, x):
        have_linear = False
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        # Max pool 1
        out = self.maxpool1(out)
        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)
        # Convolution 3 
        out = self.cnn3(out)
        out = self.relu3(out)
        # Max pool 2 
        out = self.maxpool2(out)
        # Average pool
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        # print(out.size())
        # Linear function (readout)
        if have_linear:
            out = self.fc1(out)
        return out

In [18]:
# load only works if the same model is already declared
model = torch.load("model_90.pt")
model.eval()

CNN_Model(
  (cnn1): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn2): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1))
  (relu2): ReLU()
  (cnn3): Conv2d(4, 10, kernel_size=(3, 3), stride=(1, 1))
  (relu3): ReLU()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (avgpool): AvgPool2d(kernel_size=4, stride=4, padding=0)
)

In [19]:
input_shape = (-1, 1, 28, 28)
verify_model(model, input_shape, verify_loader)

tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 5, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 2, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 9,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0,
        2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 3, 3, 1, 8,
        1, 7, 0, 9])
tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5,
        4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2,
        4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0,
        2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4,
        1, 7, 6, 9])
--------------
tensor([6, 0, 5, 4, 2, 9, 2, 1, 9, 4, 8, 1, 3, 9, 7, 4, 4, 4, 9, 2, 5, 6, 7, 6,
        9, 9, 0, 5, 8, 5, 6, 6, 5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2,
        0, 3, 9, 5, 5, 5, 1, 5, 6, 0, 5, 9, 4, 6, 5, 4, 6, 5, 8, 3, 1, 4, 4, 7,
        2, 3, 2, 1, 1, 8, 1, 8, 1, 8, 5, 0, 3, 9, 2, 5, 0, 1, 1

In [20]:
from PIL import Image
import numpy as np

In [21]:
img = Image.open("pics/file_6_1.png")
img.load()
img = np.asarray( img, dtype="int32" )
img = img.astype('float32') / 255
img = torch.from_numpy(img)
img = img.unsqueeze(0)

In [22]:
out = model(img)

In [23]:
print(out.size())
# out = out.squeeze(1)
# print(out.size())
print(out)
torch.max(out, 0)

torch.Size([10, 1])
tensor([[ 1.2902],
        [11.7298],
        [ 3.9061],
        [ 7.7338],
        [ 2.3796],
        [ 0.6430],
        [ 3.1881],
        [ 0.6020],
        [ 1.9333],
        [ 6.8049]], grad_fn=<ViewBackward0>)


torch.return_types.max(
values=tensor([11.7298], grad_fn=<MaxBackward0>),
indices=tensor([1]))

In [84]:
parameters = {}
parameters_Q15_16 = {}

In [85]:
for name, param in model.named_parameters():
    parameters[name] = param.data.tolist()

In [86]:
for name in parameters:
    reorganize = []
    raw = []
    if "weight" in name:
        for per_o_channel in parameters[name]:
            per_o = []
            per_o_raw = []
            for per_in_channel in per_o_channel:
                per_in = []
                per_in_raw = []
                for weights in per_in_channel:
                    for weight in weights:
                        per_in.append(weight)
                        per_in_raw.append(int(weight * (2**16)))
                per_o.append(per_in)
                per_o_raw.append(per_in_raw)
            reorganize.append(per_o)
            raw.append(per_o_raw)
        parameters[name] = reorganize
        parameters_Q15_16[''.join(name.split('.'))] = raw
    else:
        for bias in parameters[name]:
            raw.append(int(bias * (2**16)))
        parameters_Q15_16[''.join(name.split('.'))] = raw
        

In [87]:
for name in parameters_Q15_16:
    print(name)
    print(parameters_Q15_16[name])
        

cnn1weight
[[[-10174, -104150, -91946, 48411, -40553, -15014, 44825, 112022, 81093]], [[194725, 122059, 46556, 115785, 24132, 50906, 13227, -3240, 4675]], [[55276, 45645, 40350, -63657, 40389, 455, 46677, 65183, 111589]], [[94716, -55682, -121383, 119082, 11150, -140981, 59706, 111818, -32214]]]
cnn1bias
[-5046, 1505, -1722, -23765]
cnn2weight
[[[109236, 2658, -11465, 42538, -58894, -44536, 31282, -104144, 142203], [-27440, -7070, 7623, 75575, 11236, -1996, -7523, -7632, 28409], [-38722, -24599, 5760, 36098, 22751, 12006, -9661, 66, 11601], [49363, -13792, 25063, -10624, 10856, 60251, 71402, 53942, 43235]], [[38758, 54588, 2798, -63034, 19660, -17945, -49817, 4620, -66929], [-254693, -19759, -112019, -27409, 46808, -6405, -521, 80937, 32075], [-98908, -14719, -76878, 19288, 16883, 251, 65486, 90184, 5513], [13370, -14866, 709, 47066, -18351, 35250, 71029, 27217, 42037]], [[-161305, -17371, -175867, -163285, 23718, -27367, -162841, -34212, -115626], [14814, -8460, -47958, 91586, -48527,

In [90]:
for name in parameters_Q15_16:
    o_ch = len(parameters_Q15_16[name])
    layer_name = name[0:4].upper()
    if "weight" in name:
        in_ch = len(parameters_Q15_16[name][0])
        print("localparam %s_IN_CH = %d;" % (layer_name, in_ch))
        print("localparam %s_OUT_CH = %d;" % (layer_name, o_ch))
        gen = "localparam [32*%s_OUT_CH*%s_IN_CH*9-1:0] PARAM" % (layer_name, layer_name)
        gen = gen + name.upper() + " = {"
        for out_channel in parameters_Q15_16[name]:
            for in_channel in out_channel:
                for weight in in_channel:
                    if weight < 0:
                        gen = gen + '-'
                    gen = gen + "32'd" + str(abs(weight)) + ", "
        gen = gen[:-2] + "};"
        print(gen)
        print("logic signed [32-1:0] %s[%s_OUT_CH][%s_IN_CH][3][3];" % (name, layer_name, layer_name))
    elif "bias" in name:
        gen = "localparam [32*" + layer_name + "_OUT_CH-1:0] PARAM" + name.upper() + " = {"
        for weight in parameters_Q15_16[name]:
            if weight < 0:
                gen = gen + '-'
            gen = gen + "32'd" + str(abs(weight)) + ", "
        gen = gen[:-2] + "};"
        print(gen)
        print("logic signed [32-1:0] %s[%s_OUT_CH];" % (name, layer_name))
    else:
        print("unexpected name: %s" % name)

localparam CNN1_IN_CH = 1;
localparam CNN1_OUT_CH = 4;
localparam [32*CNN1_OUT_CH*CNN1_IN_CH*9-1:0] PARAMCNN1WEIGHT = {-32'd10174, -32'd104150, -32'd91946, 32'd48411, -32'd40553, -32'd15014, 32'd44825, 32'd112022, 32'd81093, 32'd194725, 32'd122059, 32'd46556, 32'd115785, 32'd24132, 32'd50906, 32'd13227, -32'd3240, 32'd4675, 32'd55276, 32'd45645, 32'd40350, -32'd63657, 32'd40389, 32'd455, 32'd46677, 32'd65183, 32'd111589, 32'd94716, -32'd55682, -32'd121383, 32'd119082, 32'd11150, -32'd140981, 32'd59706, 32'd111818, -32'd32214};
logic signed [32-1:0] cnn1weight[CNN1_OUT_CH][CNN1_IN_CH][3][3];
localparam [32*CNN1_OUT_CH-1:0] PARAMCNN1BIAS = {-32'd5046, 32'd1505, -32'd1722, -32'd23765};
logic signed [32-1:0] cnn1bias[CNN1_OUT_CH];
localparam CNN2_IN_CH = 4;
localparam CNN2_OUT_CH = 4;
localparam [32*CNN2_OUT_CH*CNN2_IN_CH*9-1:0] PARAMCNN2WEIGHT = {32'd109236, 32'd2658, -32'd11465, 32'd42538, -32'd58894, -32'd44536, 32'd31282, -32'd104144, 32'd142203, -32'd27440, -32'd7070, 32'd7623, 32'd75